<a href="https://colab.research.google.com/github/pratjz/Tensorflow/blob/master/8_Distributed_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and setting up the environment

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0

## Stage 2: Importing project dependencies

In [ ]:
import time
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

## Stage 3: Dataset preprocessing

### Loading the MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Image normalization

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
X_train.shape

(60000, 28, 28)

### Dataset reshaping

In [ ]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [ ]:
X_train.shape

(60000, 784)

## Stage 4: Distributed Training

### Defining normal (non distributed) model

In [ ]:
model_normal = tf.keras.models.Sequential()

In [ ]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [ ]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [ ]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [ ]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Defining the Distributed Strategy

In [ ]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


### Defining a distributed model

In [ ]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Speed comparison between normal training and distributed training process

In [ ]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tens

In [ ]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.2784 - sparse_categorical_accuracy: 0.9193
Epoch 2/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1347 - sparse_categorical_accuracy: 0.9592
Epoch 3/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1005 - sparse_categorical_accuracy: 0.9691
Epoch 4/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.0843 - sparse_categorical_accuracy: 0.9738
Epoch 5/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.0739 - sparse_categorical_accuracy: 0.9765
Epoch 6/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.0606 - sparse_categorical_accuracy: 0.9808
Epoch 7/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.0578 - sparse_categorical_accuracy: 0.9809
Epoch 8/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.0503 - sparse_categorical_accuracy: 0.9834
Epoch 9/10
2400/2400 [==========